In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
#Install the happytransformer module. 
!pip install happytransformer

     |████████████████████████████████| 225kB 7.9MB/s 
     |████████████████████████████████| 2.1MB 16.2MB/s 
     |████████████████████████████████| 1.2MB 34.7MB/s 
     |████████████████████████████████| 112kB 51.9MB/s 
     |████████████████████████████████| 245kB 49.4MB/s 
     |████████████████████████████████| 901kB 45.4MB/s 
     |████████████████████████████████| 3.3MB 55.4MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
from happytransformer import HappyTextClassification
import nltk 
nltk.download('punkt')
nltk.download('stopwords')

# DistilBertTokenizer is identical to BertTokenizer
from transformers import DistilBertModel, DistilBertTokenizer
import vsm

# set working directory
os.chdir('/content/gdrive/My Drive/AI Creativity')
os.listdir()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


["Justin's data",
 'K-Fold',
 'runs',
 'train_bin.csv',
 'df_resid.csv',
 'cross_domain_results.csv',
 'rater_number_function.csv',
 'rater_number_function.gsheet',
 'cross_domain_results.gsheet',
 '2fake_prediction.csv',
 '3fake_prediction.csv',
 '4fake_prediction.csv',
 '5fake_prediction.csv',
 '1fake_prediction.csv',
 'tmp.csv',
 'cs224u',
 '1homo_prediction.csv',
 '1scrambled_prediction.csv',
 '1regression_prediction.csv',
 '2homo_prediction.csv',
 '2regression_prediction.csv',
 '2scrambled_prediction.csv',
 '3homo_prediction.csv',
 '3scrambled_prediction.csv',
 '3regression_prediction.csv',
 '4homo_prediction.csv',
 '4regression_prediction.csv',
 '4scrambled_prediction.csv',
 '5scrambled_prediction.csv',
 '5regression_prediction.csv',
 '5homo_prediction.csv',
 'Cross_Domain_Creativity_Combined_split1_scrambled_prediction.csv',
 'Cross_Domain_Creativity_Combined_split1homo_prediction.csv',
 'Cross_Domain_Creativity_Combined_split1normal_prediction.csv',
 'Cross_Domain_Creativity_Co

In [ ]:
import random
random.seed(1)
import re


# Our dataset is an excel sheet with multiple sheets. 
# Each sheet include ideas from one sample along with ratings on several metrics 
#3 constructs (creativity, usefulness, novelty) * 3 types of judges (expert, consumers, combined)
# the following function allow us to extract a specific type of labels together with the ideas
# For unknown reasons, I received an error when using BERT-like models to process a too long sequence (Perhaps because of the limit of 512 tokens of BERT)
# The "length" argument allows us to keep ideas with less than a certain number of words. The default is 400, which works fine.
def extract_metric(study, metric, length = 400):
  df0 = pd.read_excel("Justin's data/Idea Ratings_Berg_2019_OBHDP.xlsx", sheet_name= study-1)
  df = df0[["Final_Idea", metric]].rename(columns={'Final_Idea': 'text', metric: 'label'})

  return(df.iloc[[len(x.split())< length for x in df['text']]])

# The "scramble" function is for feature masking. It scrambles a sequence by randomizing the order of words
def scramble(text):
  words = text.split()
  n = len(words)
  scrambled = random.sample(words, n)
  return(" ".join(scrambled))

# word_rep and homogenize are used for feature masking. The "homogenize" function replace words 
# with an arbitrarily chose word with the same length or similar length.

def word_rep(word):
  if len(word) == 1:
    word = "a"
  if len(word) == 2:
    word = "an"
  if len(word) == 3:
    word = "and"
  if len(word) == 4:
    word = "andy"
  if len(word) == 5:
    word = "antic"
  if len(word) == 6:
    word = "accent"
  if len(word) == 7:
    word = "ancient"
  if len(word) == 8:
    word = "accident"
  if len(word) == 9:
    word = "accidents"
  if len(word) == 10:
    word = "accidental"
  if len(word) >= 11:
    word = "accidentally"
  return(word)
def homogenize(text):
  words = text.split()
  homogenized = [word_rep(word) for word in words]
  return(" ".join(homogenized))




In [ ]:
# Domain General Predictions

for metric in ["Creativity_Combined", "Novelty_Combined", "Usefulness_Combined"]: # loop through the three constructs
  
  # In domain general predictions, we combine the three samples as our corpus
  train_file= pd.concat([extract_metric(1,metric),
            extract_metric(2,metric),
            extract_metric(3,metric)]).reset_index()

  # Roughly split the dataset into 5 subsets for cross-validation. 
  # I believe there are more efficient way to do this, but this is how I often do in R.
  n = train_file.shape[0]
  index = np.random.randint(1,6,n)
  for i in range(1,6):
    #Split the dataset into train set and test set.
    train_set = train_file[index != i]
    # The happytransformer function takes filenames as input. 
    # Therefore, I write the train set and the test set into csv files
    train_set.to_csv("train_file.csv")
    test_set = train_file[index == i]
    test_set.to_csv("test_file.csv")

    #Instantiate a BERT-like model.
    # Set num_labels = 1 to make the model run regressions
    model = HappyTextClassification(
        #"ROBERTA", "roberta-base",
        "DISTILBERT", "distilbert-base-uncased",
        #"XLNET", "xlnet-base-cased",
        #"ALBERT", "albert-base-v2",
                                  num_labels=1)
    model.train("train_file.csv")

    # create a copy of the test set and scramble it
    scrambled = test_set.copy()
    scrambled['text'] = [scramble(x) for x in test_set.text]
    scrambled.to_csv("scrambled.csv")

    # Create a copy of the test set and homogenize it
    homo = test_set.copy()
    homo['text'] = [homogenize(x) for x in test_set.text]
    homo.to_csv("homo.csv")

    # the test method of happytransformer models return predicted labels and scores.
    # Because we set the num_labels = 1, only the scores matter
    pred = model.test("test_file.csv")
    scrambled_pred = model.test("scrambled.csv")
    homo_pred = model.test("homo.csv")

    # Extract the scores
    pred_score = [x.score for x in pred]
    scrambled_pred_score = [x.score for x in scrambled_pred]
    homo_pred_score = [x.score for x in homo_pred]

    # This chunk of code might be totally redundant. 
    df_pred = test_set.copy()
    df_scrambled = scrambled.copy()
    df_homo = homo.copy()

    # Replace the original labels with the predicted scores and write the results into csv files
    #(perhaps I should have created new columns for the predicted scores)
    df_scrambled['label'] = scrambled_pred_score
    df_scrambled.to_csv("Cross_Domain_" + metric + "_split" + str(i) + '_scrambled_prediction.csv')
    df_homo['label'] = homo_pred_score
    df_homo.to_csv("Cross_Domain_" + metric + "_split" + str(i) + '_homo_prediction.csv')
    df_pred['label'] = pred_score
    df_pred.to_csv("Cross_Domain_" + metric + "_split" + str(i) + '_normal_prediction.csv')

    # Print pearson correlation for each iteration. Not important
    print("normal: "+ str(np.corrcoef(pred_score, test_set['label'])))
    print("scrambled: "+ str(np.corrcoef(scrambled_pred_score, test_set['label'])))
    print("homo: "+ str(np.corrcoef(homo_pred_score, test_set['label'])))


#IMPORTANT NOTE: the original happytransformer model only takes labels of integer values, which will lead to the following error:
# invalid literal for int() with base 10: '5.7'
# To fix this, you can open the following file (it will show up in the error message in colabratory):
#/usr/local/lib/python3.7/dist-packages/happytransformer/tc/trainer.py 
# and replace the following code with my code
# Original code
        # contexts = []
        # labels = []
        # with open(filepath, newline='') as csv_file:
        #     reader = csv.DictReader(csv_file)
        #     for row in reader:
        #         contexts.append(row['text'])
        #         if not test_data:
        #             labels.append(int(row['label']))
        # csv_file.close()
#My code
        # import pandas as pd
        # df= pd.read_csv(filepath)
        # contexts = list(df['text'])
        # labels = list(df['label'])

# After the replacement, rerun the notebook without installing the happytransformer module. It should work not
# If you import the module from a local directory, you only need to do this once. 
# However, in colaboratory, I have to repeat this step every time.

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

05/09/2021 05:21:14 - INFO - happytransformer.happy_transformer -   Using model: cuda


05/09/2021 05:21:20 - INFO - happytransformer.happy_transformer -   Preprocessing dataset...


ValueError: ignored

In [ ]:
general_results = {}

for metric in ["Creativity_Combined", "Novelty_Combined", "Usefulness_Combined"]:
  for pred_type in ['normal', "_scrambled", "homo"]:
    #combine all predicted scores to get the predictions of the whole dataset
    all_filenames = ["Cross_Domain_" + metric + "_split" + str(i)+ pred_type +"_prediction.csv" for i in range(1,6)]
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames]) 

    # Calculate the Pearson correlation between the predicted scores and human ratings
    general_results[metric+ '_' + pred_type] = (np.corrcoef(combined_csv.sort_values(by='Unnamed: 0')['label'], train_file['label'])[0,1])

# Write out the results into csv files. I use these files to make plots in R
pd.DataFrame.from_dict({'key':general_results.keys(), 'value': general_results.values()}).to_csv("Domain_general_results(0503).csv")


NameError: ignored

In [ ]:
# Domain Specific Predictions
for metric in ["Creativity_Combined", "Novelty_Combined", "Usefulness_Combined"]:
  for study in [1,2,3]:
    
    # Everything is the same as in domain general predictions, except that each sample is used separately here
    train_file= extract_metric(study,metric).sample(n = 250).reset_index()
    n = train_file.shape[0]
    index = np.random.randint(1,6,n)
    for i in range(1,6):
      train_set = train_file[index != i]
      train_set.to_csv("train_file.csv")
      test_set = train_file[index == i]
      test_set.to_csv("test_file.csv")

      model = HappyTextClassification(
          #"ROBERTA", "roberta-base",
          "DISTILBERT", "distilbert-base-uncased",
          #"XLNET", "xlnet-base-cased",
          #"ALBERT", "albert-base-v2",
                                    num_labels=1)
      model.train("train_file.csv")

      scrambled = test_set.copy()
      homo = test_set.copy()

      scrambled['text'] = [scramble(x) for x in test_set.text]
      homo['text'] = [homogenize(x) for x in test_set.text]

      scrambled.to_csv("scrambled.csv")
      homo.to_csv("homo.csv")

      pred = model.test("test_file.csv")
      scrambled_pred = model.test("scrambled.csv")
      homo_pred = model.test("homo.csv")

      pred_score = [x.score for x in pred]
      scrambled_pred_score = [x.score for x in scrambled_pred]
      homo_pred_score = [x.score for x in homo_pred]

      df_pred = test_set.copy()
      df_scrambled = scrambled.copy()
      df_homo = homo.copy()

      df_scrambled['label'] = scrambled_pred_score
      df_scrambled.to_csv("Domain_specific_study" + str(study) + "_" + metric + "_split" + str(i) + '_scrambled_prediction.csv')
      df_homo['label'] = homo_pred_score
      df_homo.to_csv("Domain_specific_study" + str(study) + "_" + metric + "_split" + str(i) + 'homo_prediction.csv')
      df_pred['label'] = pred_score
      df_pred.to_csv("Domain_specific_study" + str(study) + "_" + metric + "_split" + str(i) + 'normal_prediction.csv')

      print("normal: "+ str(np.corrcoef(pred_score, test_set['label'])))
      print("scrambled: "+ str(np.corrcoef(scrambled_pred_score, test_set['label'])))
      print("homo: "+ str(np.corrcoef(homo_pred_score, test_set['label'])))


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Step,Training Loss
500,0.693900


100%|██████████| 51/51 [00:00<00:00, 182.80it/s]


normal: [[1.         0.65939717]
 [0.65939717 1.        ]]
scrambled: [[1.         0.60815799]
 [0.60815799 1.        ]]
homo: [[1.         0.42781806]
 [0.42781806 1.        ]]


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

...

normal: [[1.         0.51215589]
 [0.51215589 1.        ]]
scrambled: [[1.         0.59781985]
 [0.59781985 1.        ]]
homo: [[1.         0.35087261]
 [0.35087261 1.        ]]


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Step,Training Loss
500,0.580200


100%|██████████| 57/57 [00:00<00:00, 165.01it/s]


normal: [[1.         0.65407312]
 [0.65407312 1.        ]]
scrambled: [[1.         0.63571671]
 [0.63571671 1.        ]]
homo: [[1.         0.42945509]
 [0.42945509 1.        ]]


In [ ]:
specific_results = {}

# It is likely that the results of happytransformer models are normalized using a sigmoid function
# Using this invers_sigmoid function allows us to get the raw scores of regression models
# Doing this seems to improve accuracy slightly
def inverse_sigmoid(y):
  return np.log(y/(1-y))

for metric in ["Creativity_Combined", "Novelty_Combined", "Usefulness_Combined"]:
  for study in [1,2,3]:
    for pred_type in ['normal', "_scrambled", "homo"]:
      all_filenames = ["Domain_specific_study" + str(study) + "_" + metric + "_split" + str(i) + pred_type+'_prediction.csv' for i in range(1,6)]
      combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames]).sort_values(by='Unnamed: 0')
      train_file= extract_metric(study,metric)

      y_estimate = combined_csv['label']
      y_true = train_file.loc[combined_csv['index'],'label']

      #specific_results[metric + "_study" + str(study)+ '_' + pred_type] = (np.corrcoef(y_estimate, y_true)[0,1])
      specific_results[metric + "_study" + str(study)+ '_' + pred_type] = (np.corrcoef(inverse_sigmoid(y_estimate), y_true)[0,1])

# pd.DataFrame.from_dict({'key':specific_results.keys(), 'value': specific_results.values()}).to_csv("Domain_specific_results(0503).csv")


Inverse_sigmoid results

{'Creativity_Combined_study1__scrambled': 0.6036988186771806,
 'Creativity_Combined_study1_homo': 0.4823619568751188,
 'Creativity_Combined_study1_normal': 0.6463445792796637,
 'Creativity_Combined_study2__scrambled': 0.7203512372807368,
 'Creativity_Combined_study2_homo': 0.5198173668616751,
 'Creativity_Combined_study2_normal': 0.7117417651112722,
 'Creativity_Combined_study3__scrambled': 0.6659989029153098,
 'Creativity_Combined_study3_homo': 0.38233451080530717,
 'Creativity_Combined_study3_normal': 0.6872549785663923,
 'Novelty_Combined_study1__scrambled': 0.5592626273318138,
 'Novelty_Combined_study1_homo': 0.49202069997900133,
 'Novelty_Combined_study1_normal': 0.5744742944075424,
 'Novelty_Combined_study2__scrambled': 0.6418145060710528,
 'Novelty_Combined_study2_homo': 0.45258013837462935,
 'Novelty_Combined_study2_normal': 0.5910802973383661,
 'Novelty_Combined_study3__scrambled': 0.5311424200434418,
 'Novelty_Combined_study3_homo': 0.3781024469416406,
 'Novelty_Combined_study3_normal': 0.5513141301276988,
 'Usefulness_Combined_study1__scrambled': 0.4715331178125642,
 'Usefulness_Combined_study1_homo': 0.36764497672426866,
 'Usefulness_Combined_study1_normal': 0.5450251915363286,
 'Usefulness_Combined_study2__scrambled': 0.7407053166149862,
 'Usefulness_Combined_study2_homo': 0.21543484128261622,
 'Usefulness_Combined_study2_normal': 0.7717636916972684,
 'Usefulness_Combined_study3__scrambled': 0.5987181160598203,
 'Usefulness_Combined_study3_homo': 0.3271331810590989,
 'Usefulness_Combined_study3_normal': 0.591836376949932}

Raw results
 {'Creativity_Combined_study1__scrambled': 0.5771698758149193,
 'Creativity_Combined_study1_homo': 0.4300978918035641,
 'Creativity_Combined_study1_normal': 0.6463088483610605,
 'Creativity_Combined_study2__scrambled': 0.701350483161944,
 'Creativity_Combined_study2_homo': 0.5321629182040738,
 'Creativity_Combined_study2_normal': 0.6961954262707623,
 'Creativity_Combined_study3__scrambled': 0.6273087736456405,
 'Creativity_Combined_study3_homo': 0.3786845560571378,
 'Creativity_Combined_study3_normal': 0.67867499963009,
 'Novelty_Combined_study1__scrambled': 0.5331403579091303,
 'Novelty_Combined_study1_homo': 0.4534316982609445,
 'Novelty_Combined_study1_normal': 0.5456892636937443,
 'Novelty_Combined_study2__scrambled': 0.5912638382707073,
 'Novelty_Combined_study2_homo': 0.4668422177205251,
 'Novelty_Combined_study2_normal': 0.539181017446131,
 'Novelty_Combined_study3__scrambled': 0.4840568557620448,
 'Novelty_Combined_study3_homo': 0.37172995291699334,
 'Novelty_Combined_study3_normal': 0.5097986565921832,
 'Usefulness_Combined_study1__scrambled': 0.47613903003544167,
 'Usefulness_Combined_study1_homo': 0.34990235954121973,
 'Usefulness_Combined_study1_normal': 0.524757089413579,
 'Usefulness_Combined_study2__scrambled': 0.7317236140363812,
 'Usefulness_Combined_study2_homo': 0.2474972202827062,
 'Usefulness_Combined_study2_normal': 0.7613030946034497,
 'Usefulness_Combined_study3__scrambled': 0.5717617988538715,
 'Usefulness_Combined_study3_homo': 0.3270518133572374,
 'Usefulness_Combined_study3_normal': 0.5761344859890838}

In [ ]:
#Here, I try to combine BERT's prediction with predictions based on other features, but it didn't work well
# You may want to ignore these for now.

all_filenames = [str(i)+"regression_prediction.csv" for i in range(1,6)]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames]).sort_values(by='Unnamed: 0').reset_index()


def get_length(df):
  nrow = df.shape[0]
  lengths = []
  characters = []
  average_len = []
  for i in range(nrow):
      lengths.append(len(df['text'][i].split()))
      characters.append(len(df['text'][i]))
      average_len.append(len(df['text'][i])/len(df['text'][i].split()))
  df_len = pd.DataFrame({'lengths': lengths,
                        'characters': characters,
                        'average_len': average_len})
  return(df_len.to_numpy())

def glove2dict(src_filename):
    """
    GloVe vectors file reader.

    Parameters
    ----------
    src_filename : str
        Full path to the GloVe file to be processed.

    Returns
    -------
    dict
        Mapping words to their GloVe vectors as `np.array`.

    """
    # This distribution has some words with spaces, so we have to
    # assume its dimensionality and parse out the lines specially:
    if '840B.300d' in src_filename:
        line_parser = lambda line: line.rsplit(" ", 300)
    else:
        line_parser = lambda line: line.strip().split()
    data = {}
    with open(src_filename, encoding='utf8') as f:
        while True:
            try:
                line = next(f)
                line = line_parser(line)
                data[line[0]] = np.array(line[1: ], dtype=np.float)
            except StopIteration:
                break
            except UnicodeDecodeError:
                pass
    return data

# glove_dict = glove2dict(
#     os.path.join('cs224u','data', 'glove.6B', 'glove.6B.300d.txt'))

def get_distance(word1, word2):
  from scipy.spatial.distance import cosine
  v1 = glove_dict[word1],
  v2 = glove_dict[word2]

  return(cosine(v1, v2))

def get_vector_distance(text):
  words = nltk.word_tokenize(text)
  stop_words = nltk.corpus.stopwords.words('english')
  words = [w for w in words if not w in stop_words] 
  n = len(words)
  distances = []
  for i in range(n):
    for j in range(i+1, n):
      if words[i] in glove_dict and words[j] in glove_dict:
        distances.append(get_distance(words[i], words[j]))
      else:
        continue
  return(distances)


def get_df_distance(df):
  lst_distances = [get_vector_distance(text) for text in df['text']]

  df_dis = pd.DataFrame({'average': [np.average(distances) for distances in lst_distances],
                    'max': [np.max(distances) for distances in lst_distances],
                    'min': [np.min(distances) for distances in lst_distances]})
  return(df_dis.to_numpy())
bert_pred = combined_csv['label']
df_length = get_length(combined_csv)
df_distance = get_df_distance(combined_csv)


0.5985653734163079

In [ ]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
X = np.column_stack([bert_pred,df_length,df_distance])
y = train_file['label'].to_numpy().reshape(-1,1)
lr.fit(X,y)
lr.score(X,y)

0.5819160869112776

In [ ]:
#Trying to use two sample two predict the other sample
train_set = pd.concat([extract_metric(1,"Novelty_Consumer"),
          extract_metric(2,"Novelty_Consumer")]).reset_index()

train_set.to_csv("train_file.csv")
test_set = extract_metric(3,"Novelty_Consumer")
test_set.to_csv("test_file.csv")

model = HappyTextClassification(
    #"ROBERTA", "roberta-base",
    "DISTILBERT", "distilbert-base-uncased",
    #"ALBERT", "albert-base-v2",
                              num_labels=1)
model.train("train_file.csv")
pred = model.test("test_file.csv")

pred_score = [x.score for x in pred]

df_pred = test_set.copy()
df_pred['label'] = pred_score
print(np.corrcoef(pred_score, test_set['label']))


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Step,Training Loss


100%|██████████| 308/308 [00:15<00:00, 19.65it/s]

[[1.         0.58764012]
 [0.58764012 1.        ]]
